# Playground for Recommeder System slides

- Stephen W. Thomas
- Used for MMA 869, MMAI 869, and GMMA 869

In [1]:
import datetime
print(datetime.datetime.now())

2019-04-23 08:57:36.395382


In [2]:
import pandas as pd
pd.show_versions(as_json=False)


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.4.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.23.4
pytest: 3.3.2
pip: 19.0.3
setuptools: 38.4.0
Cython: 0.27.3
numpy: 1.15.4
scipy: 1.0.0
pyarrow: None
xarray: None
IPython: 6.2.1
sphinx: 1.6.6
patsy: 0.5.0
dateutil: 2.6.1
pytz: 2017.3
blosc: None
bottleneck: 1.2.1
tables: 3.4.2
numexpr: 2.6.6
feather: None
matplotlib: 3.0.0
openpyxl: 2.4.10
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.0.2
lxml: 4.1.1
bs4: 4.6.0
html5lib: 1.0.1
sqlalchemy: 1.2.1
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.metrics import silhouette_score, silhouette_samples
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import itertools

import scipy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Manual Examples

In [4]:
u = pd.DataFrame(data={'Gladiator': [4, 5, 4, 0, 0, 0], 
                       'Devil Wears Prada': [3, 0, 0, 3, 4, 0],
                       'Matrix': [0, 4, 5, 0, 0, 2],
                       'Lord of the Rings': [4, 0, 3, 0, 0, 4],
                       'Saving Private Ryan': [5, 4, 4, 0, 0, 0],
                       'Mean Girls': [0, 0, 0, 5, 4, 5],})
u

,Gladiator,Devil Wears Prada,Matrix,Lord of the Rings,Saving Private Ryan,Mean Girls
0,4,3,0,4,5,0
1,5,0,4,0,4,0
2,4,0,5,3,4,0
3,0,3,0,0,0,5
4,0,4,0,0,0,4
5,0,0,2,4,0,5


In [5]:
from sklearn.metrics.pairwise import *

# Computes similarity between all rows/observations in a dataframe
np.around(cosine_similarity(u), 2)

array([[1.  , 0.65, 0.73, 0.19, 0.26, 0.29],
       [0.65, 1.  , 0.91, 0.  , 0.  , 0.16],
       [0.73, 0.91, 1.  , 0.  , 0.  , 0.4 ],
       [0.19, 0.  , 0.  , 1.  , 0.97, 0.64],
       [0.26, 0.  , 0.  , 0.97, 1.  , 0.53],
       [0.29, 0.16, 0.4 , 0.64, 0.53, 1.  ]])

In [6]:
# Computes similarity between all rows/observations in a dataframe
np.around(cosine_distances(u), 2)

array([[0.  , 0.35, 0.27, 0.81, 0.74, 0.71],
       [0.35, 0.  , 0.09, 1.  , 1.  , 0.84],
       [0.27, 0.09, 0.  , 1.  , 1.  , 0.6 ],
       [0.81, 1.  , 1.  , 0.  , 0.03, 0.36],
       [0.74, 1.  , 1.  , 0.03, 0.  , 0.47],
       [0.71, 0.84, 0.6 , 0.36, 0.47, 0.  ]])

In [7]:
np.around(cosine_similarity(u.transpose()), 2)

array([[1.  , 0.27, 0.79, 0.58, 0.98, 0.  ],
       [0.27, 1.  , 0.  , 0.32, 0.34, 0.65],
       [0.79, 0.  , 1.  , 0.54, 0.71, 0.18],
       [0.58, 0.32, 0.54, 1.  , 0.66, 0.38],
       [0.98, 0.34, 0.71, 0.66, 1.  , 0.  ],
       [0.  , 0.65, 0.18, 0.38, 0.  , 1.  ]])

In [8]:
np.around(euclidean_distances(u), 2)

array([[ 0.  ,  6.56,  6.  ,  9.06,  8.6 ,  8.89],
       [ 6.56,  0.  ,  3.32,  9.54,  9.43,  9.27],
       [ 6.  ,  3.32,  0.  , 10.  ,  9.9 ,  8.19],
       [ 9.06,  9.54, 10.  ,  0.  ,  1.41,  5.39],
       [ 8.6 ,  9.43,  9.9 ,  1.41,  0.  ,  6.08],
       [ 8.89,  9.27,  8.19,  5.39,  6.08,  0.  ]])

# Jaccard

In [9]:
def jaccard_similarity(l1, l2):
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection / union)

list1 = ['dog', 'cat', 'fish']
list2 = ['dog', 'cat', 'horse', 'pig']
jaccard_similarity(list1, list2)

0.4

# Surprise Package

In [10]:
from surprise import SVD
from surprise import Dataset
from surprise import get_dataset_dir


# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

In [11]:
import io  # needed because of weird encoding of u.item file

def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Look at 10 random elements in the dict
list(itertools.islice(name_to_rid.items(), 20))

[('Toy Story (1995)', '1'),
 ('GoldenEye (1995)', '2'),
 ('Four Rooms (1995)', '3'),
 ('Get Shorty (1995)', '4'),
 ('Copycat (1995)', '5'),
 ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', '6'),
 ('Twelve Monkeys (1995)', '7'),
 ('Babe (1995)', '8'),
 ('Dead Man Walking (1995)', '9'),
 ('Richard III (1995)', '10'),
 ('Seven (Se7en) (1995)', '11'),
 ('Usual Suspects, The (1995)', '12'),
 ('Mighty Aphrodite (1995)', '13'),
 ('Postino, Il (1994)', '14'),
 ("Mr. Holland's Opus (1995)", '15'),
 ('French Twist (Gazon maudit) (1995)', '16'),
 ('From Dusk Till Dawn (1996)', '17'),
 ('White Balloon, The (1995)', '18'),
 ("Antonia's Line (1995)", '19'),
 ('Angels and Insects (1995)', '20')]

In [12]:
type(data)

surprise.dataset.DatasetAutoFolds

In [13]:
type(data.raw_ratings)

# Format is:
# user item rating timestamp
data.raw_ratings[0]
data.raw_ratings[1]

list

('196', '242', 3.0, '881250949')

('186', '302', 3.0, '891717742')

In [14]:
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])

In [15]:
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [16]:
item_names = pd.DataFrame.from_dict(rid_to_name, orient="index", columns=['item_name'])
item_names['item'] = item_names.index
item_names.head()
df = pd.merge(df, item_names)
df.head()

,item_name,item
1,Toy Story (1995),1
2,GoldenEye (1995),2
3,Four Rooms (1995),3
4,Get Shorty (1995),4
5,Copycat (1995),5


,user,item,rating,timestamp,item_name
0,196,242,3.0,881250949,Kolya (1996)
1,63,242,3.0,875747190,Kolya (1996)
2,226,242,5.0,883888671,Kolya (1996)
3,154,242,3.0,879138235,Kolya (1996)
4,306,242,5.0,876503793,Kolya (1996)


In [17]:
df[df['user'] == '22']

,user,item,rating,timestamp,item_name
414,22,377,1.0,878887116,Heavyweights (1994)
1044,22,265,3.0,878888066,"Hunt for Red October, The (1990)"
1180,22,451,4.0,878887062,Grease (1978)
1901,22,222,4.0,878887925,Star Trek: First Contact (1996)
2288,22,29,1.0,878888228,Batman Forever (1995)
2978,22,144,5.0,878887680,Die Hard (1988)
3141,22,118,4.0,878887983,Twister (1996)
3940,22,546,3.0,878888107,Broken Arrow (1996)
5653,22,194,5.0,878886607,"Sting, The (1973)"
6655,22,201,4.0,878886449,Evil Dead II (1987)


In [18]:
import pandas_profiling

pandas_profiling.ProfileReport(df, check_correlation=False)

C:\Users\st50\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas_profiling\plot.py:15: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use(BACKEND)


Number of variables,5
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,4.6 MiB
Average record size in memory,48.0 B
Numeric,1
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# Build the Recommender System

In [21]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, SVD

trainset, testset = train_test_split(data, test_size=.25)

In [22]:
sim_options = {'name': 'MSD',
               'user_based': True  # compute  similarities between items
               }

algo = KNNBasic(k=10, min_k=4, sim_options=sim_options)
%time algo.fit(trainset)
%time predictions = algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Wall time: 481 ms


Wall time: 2.38 s


In [23]:
accuracy.rmse(predictions)

RMSE: 0.9938


0.993794988250414

In [24]:
predictions[0:10]

[Prediction(uid='435', iid='825', r_ui=3.0, est=2.9004180006145597, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='450', iid='702', r_ui=4.0, est=2.9904852343595913, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='524', iid='210', r_ui=3.0, est=3.8999271811545055, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='893', iid='410', r_ui=4.0, est=3.586319472953688, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='697', iid='1059', r_ui=2.0, est=2.8860960337137107, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='655', iid='962', r_ui=5.0, est=3.0991588559692618, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='707', iid='632', r_ui=4.0, est=3.84383353055478, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='630', iid='983', r_ui=3.0, est=2.228903100536477, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='495', iid='182', r_ui=5.0,

# Predict for a specific user/item pair

In [25]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)
pred

user: 196        item: 302        r_ui = None   est = 4.10   {'actual_k': 10, 'was_impossible': False}


Prediction(uid='196', iid='302', r_ui=None, est=4.1, details={'actual_k': 10, 'was_impossible': False})

# Get Top-Predicted Items for a Specific User

In [26]:
from collections import defaultdict

def get_top_n(algo, predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        #rid = algo.trainset.to_raw_iid(iid)
        item_name = rid_to_name[iid]
        #print('uid={}, iid={}, item_name={}, est={}'.format(uid, iid, item_name, est))
        top_n[uid].append((iid, item_name, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[2], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_user_neighbors(uid, k=10):
    # Retrieve inner id of the movie
    #raw_id = name_to_rid[item_name]
    #inner_id = algo.trainset.to_inner_iid(raw_id)

    # Retrieve inner ids of the nearest neighbors of the movie.
    neighbors = algo.get_neighbors(uid, k=10)

    # Convert inner ids of the neighbors into names.
    #neighbors = (algo.trainset.to_raw_iid(inner_id) for inner_id in neighbors)
    #neighbors = (rid_to_name[rid] for rid in neighbors)
    return neighbors

def user_info(uid, df, top_n, predictions):
    print('Everything we know about User {}'.format(uid))
    
    df_user = df[df['user'] == uid]
    print('Rating stats:')
    print(df_user['rating'].value_counts(sort=False).sort_index())
    
    print('\nTop 10 movies:')
    print(df_user.sort_values(by=['rating'], ascending=False).head(5))
    
    print('\nBottom 10 movies:')
    print(df_user.sort_values(by=['rating'], ascending=True).head(5))
    
    print('\nTop 10 predictions:')
    for iid, item_name, rating in top_n[uid]:
        print("{}, {}, {}".format(iid, item_name, rating))

In [27]:
top_n = get_top_n(algo, predictions, n=10)

In [28]:
user_info(str(653), df, top_n, predictions)

Everything we know about User 653
Rating stats:
1.0    50
2.0    82
3.0    77
4.0    53
5.0    21
Name: rating, dtype: int64

Top 10 movies:
      user item  rating  timestamp                 item_name
50926  653   50     5.0  878854100          Star Wars (1977)
46094  653  188     5.0  878854145  Full Metal Jacket (1987)
71080  653  180     5.0  878854593     Apocalypse Now (1979)
29817  653  127     5.0  878853780     Godfather, The (1972)
70088  653  239     5.0  878854475           Sneakers (1992)

Bottom 10 movies:
      user  item  rating  timestamp  \
78597  653   571     1.0  880153406   
92263  653   638     1.0  878866636   
47883  653   407     1.0  878867398   
90863  653  1183     1.0  880153329   
90500  653   136     1.0  880149965   

                                               item_name  
78597                            Another Stakeout (1993)  
92263  Return of Martin Guerre, The (Retour de Martin...  
47883                                    Spy Hard (1996)  
908

In [29]:
user_info(str(2), df, top_n, predictions)

Everything we know about User 2
Rating stats:
1.0     4
2.0     1
3.0    17
4.0    27
5.0    13
Name: rating, dtype: int64

Top 10 movies:
      user item  rating  timestamp                      item_name
31       2  242     5.0  888552084                   Kolya (1996)
84916    2  311     5.0  888552084  Wings of the Dove, The (1997)
66348    2  285     5.0  888552084          Secrets & Lies (1996)
74975    2  316     5.0  888979693      As Good As It Gets (1997)
72933    2  283     5.0  888552084                    Emma (1996)

Bottom 10 movies:
      user item  rating  timestamp  \
89687    2  314     1.0  888980085   
78824    2  309     1.0  888980029   
15258    2  294     1.0  888551648   
47094    2  315     1.0  888550774   
29561    2   10     2.0  888551853   

                                         item_name  
89687  3 Ninjas: High Noon At Mega Mountain (1998)  
78824                              Deceiver (1997)  
15258                             Liar Liar (1997)  
47094

In [30]:
user_info(str(8), df, top_n, predictions)

Everything we know about User 8
Rating stats:
1.0     5
2.0     4
3.0    10
4.0    19
5.0    21
Name: rating, dtype: int64

Top 10 movies:
      user item  rating  timestamp                            item_name
2138     8  222     5.0  879362356      Star Trek: First Contact (1996)
45967    8  188     5.0  879362356             Full Metal Jacket (1987)
28892    8  228     5.0  879362286  Star Trek: The Wrath of Khan (1982)
29676    8  127     5.0  879362123                Godfather, The (1972)
33226    8   56     5.0  879362183                  Pulp Fiction (1994)

Bottom 10 movies:
      user item  rating  timestamp                        item_name
29509    8  688     1.0  879361732        Leave It to Beaver (1997)
48523    8  259     1.0  879361604      George of the Jungle (1997)
81391    8  687     1.0  879361825             McHale's Navy (1997)
84270    8  457     1.0  879361825  Free Willy 3: The Rescue (1997)
19095    8  385     1.0  879362124                 True Lies (1994)

T

In [31]:
# Print the recommended items for all users
#for uid, user_ratings in top_n.items():
    #print(uid, [iid for (iid, _) in user_ratings])

# Get Nearest Neighbors of an Item

In [32]:
def get_item_neighbors(item_name, k=10):
    # Retrieve inner id of the movie
    raw_id = name_to_rid[item_name]
    inner_id = algo.trainset.to_inner_iid(raw_id)

    # Retrieve inner ids of the nearest neighbors of the movie.
    neighbors = algo.get_neighbors(inner_id, k=10)

    # Convert inner ids of the neighbors into names.
    neighbors = (algo.trainset.to_raw_iid(inner_id) for inner_id in neighbors)
    neighbors = (rid_to_name[rid] for rid in neighbors)
    return neighbors

item_name = 'Toy Story (1995)'
print('The nearest neighbors of {} are:'.format(item_name))
for movie in get_item_neighbors(item_name):
    print(movie)
    
    
item_name = 'Star Wars (1977)'
print('\nThe nearest neighbors of {} are:'.format(item_name))
for movie in get_item_neighbors(item_name):
    print(movie)

The nearest neighbors of Toy Story (1995) are:
Jerry Maguire (1996)
Executive Decision (1996)
McHale's Navy (1997)
Batman (1989)
Event Horizon (1997)
In the Line of Fire (1993)
Othello (1995)
Mr. Magoo (1997)
As Good As It Gets (1997)
Star Trek III: The Search for Spock (1984)

The nearest neighbors of Star Wars (1977) are:
It's My Party (1995)
Better Off Dead... (1985)
Grease (1978)
Get Shorty (1995)
Sleeper (1973)
Bottle Rocket (1996)
Wyatt Earp (1994)
Man Without a Face, The (1993)
Rear Window (1954)
Citizen Kane (1941)


# Cross Validation

In [34]:
from surprise.model_selection import cross_validate 
algo = SVD()

# Run 5-fold cross-validation and print results.
%time cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9351  0.9396  0.9335  0.9345  0.9345  0.9354  0.0021  
MAE (testset)     0.7386  0.7395  0.7349  0.7338  0.7380  0.7369  0.0022  
Fit time          4.23    4.06    4.73    4.49    4.49    4.40    0.23    
Test time         0.13    0.11    0.15    0.25    0.11    0.15    0.05    
Wall time: 23.5 s


{'fit_time': (4.227937698364258,
  4.06334924697876,
  4.728314638137817,
  4.490551471710205,
  4.488699913024902),
 'test_mae': array([0.7385561 , 0.73949902, 0.73487977, 0.73375237, 0.73801203]),
 'test_rmse': array([0.93511171, 0.93958069, 0.93353752, 0.93446208, 0.93448386]),
 'test_time': (0.13312673568725586,
  0.10858607292175293,
  0.15462064743041992,
  0.2460339069366455,
  0.10852813720703125)}

In [35]:
%time cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9740  0.9870  0.9773  0.9817  0.9735  0.9787  0.0051  
MAE (testset)     0.7678  0.7782  0.7736  0.7769  0.7678  0.7728  0.0044  
Fit time          0.45    0.65    0.48    0.47    0.67    0.55    0.10    
Test time         2.97    3.28    2.61    3.21    2.78    2.97    0.25    
Wall time: 18.5 s


{'fit_time': (0.4505171775817871,
  0.6505205631256104,
  0.4805014133453369,
  0.47305846214294434,
  0.6720094680786133),
 'test_mae': array([0.76776806, 0.77819349, 0.77357252, 0.77687576, 0.76777097]),
 'test_rmse': array([0.97395341, 0.98695567, 0.97730819, 0.98166202, 0.97346878]),
 'test_time': (2.9716904163360596,
  3.2802021503448486,
  2.614149332046509,
  3.211576223373413,
  2.775676965713501)}

# Grid Search

## SVD Algo

In [36]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

%time gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Wall time: 36.6 s
0.9638149011056103
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


# KNN Algo

In [37]:
param_grid_knn = {'k': [2, 10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5, 20],
                              'user_based': [False, True],
                              'verbose': [False]}
              }

gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=3)
%time gs_knn.fit(data)
print(gs_knn.best_score['rmse'])
print(gs_knn.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [39]:
gs_knn.cv_results

{'mean_fit_time': array([0.50478681, 0.31580297, 0.5062228 , 0.42185473, 0.49398247,
        0.33735625, 1.78118761, 1.61826611, 1.46062605, 1.37876765,
        1.22636962, 0.92146516, 0.56697464, 0.36447732, 0.50001518,
        0.3091356 , 0.49115515, 0.38829104, 1.91266179, 1.26995945,
        1.57101623, 1.4977626 , 1.48878749, 1.29297384, 0.71212379,
        0.48965534, 0.73597034, 0.48193439, 0.65429449, 0.43269499,
        2.19901768, 1.39046375, 1.55867418, 1.30596217, 1.424474  ,
        1.05151637]),
 'mean_test_mae': array([0.95427697, 0.89035917, 0.87975372, 0.8667584 , 0.87111232,
        0.85975428, 1.06031078, 0.94599882, 0.94120944, 0.91502693,
        0.91495209, 0.90185347, 0.81035262, 0.78900667, 0.7876052 ,
        0.78156498, 0.82150915, 0.80490103, 0.88312597, 0.82912068,
        0.83491513, 0.81889541, 0.85518452, 0.83885626, 0.78728922,
        0.7787851 , 0.78146158, 0.77536552, 0.82395513, 0.80548456,
        0.84338058, 0.81286148, 0.82334875, 0.80738881, 0.85

In [40]:
param_grid_knn = {'k': [2, 10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5, 20],
                              'user_based': [True],
                              'verbose': [False]}
              }


gs_knn = GridSearchCV(KNNWithMeans, param_grid_knn, measures=['rmse', 'mae'], cv=3)
%time gs_knn.fit(data)
print(gs_knn.best_score['rmse'])
print(gs_knn.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix.